# Getting Started: Mean Imputation

- This notebook provides a simple method to imputing missing values, and provides code for making a submission file.
- EDA revealed F_2 columns are missing no values
- all columns uncorrelated except for F_4*
- this means the best estimator for missing values for F_1,F_3 is the mean (since competition metric is RMSE)
- we only need to impute F_4*

In [1]:
from pathlib import Path

import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer
from sklearn.compose import ColumnTransformer

from xgboost import XGBRegressor

from tqdm import tqdm

RANDOM_STATE=42
INPUT_PATH = Path('./input')

C:\Users\orovi\anaconda3\envs\kaggle\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
dtypes = {'row_id' : 'int',
          'F_2_0' : 'int', 'F_2_1' : 'int', 'F_2_2' : 'int',
          'F_2_3' : 'int', 'F_2_4' : 'int', 'F_2_5' : 'int', 
          'F_2_6' : 'int', 'F_2_7' : 'int', 'F_2_8' : 'int',
          'F_2_9' : 'int', 'F_2_10' : 'int', 'F_2_11' : 'int',
          'F_2_12' : 'int', 'F_2_13' : 'int', 'F_2_14' : 'int',
          'F_2_15' : 'int', 'F_2_16' : 'int', 'F_2_17' : 'int',
          'F_2_18' : 'int', 'F_2_19' : 'int', 'F_2_20' : 'int',
          'F_2_21' : 'int', 'F_2_22' : 'int', 'F_2_23' : 'int',
          'F_2_24' : 'int'}

data = pd.read_csv(INPUT_PATH / 'data.csv', 
                   index_col='row_id',
                   dtype = dtypes)
submission = pd.read_csv(INPUT_PATH / 'sample_submission.csv', 
                         index_col='row-col')

display(data)

,F_1_0,F_1_1,F_1_2,F_1_3,F_1_4,F_1_5,F_1_6,F_1_7,F_1_8,F_1_9,...,F_4_5,F_4_6,F_4_7,F_4_8,F_4_9,F_4_10,F_4_11,F_4_12,F_4_13,F_4_14
row_id,,,,,,,,,,,,,,,,,,,,,
0,-0.354591,-0.464038,2.304115,0.734486,1.696395,0.136285,-0.518344,0.502640,-1.852504,-0.500665,...,3.744152,0.794438,0.265185,-0.561809,0.196480,0.373434,6.206995,3.809505,1.236486,1.182055
1,1.380940,-0.499626,-0.418548,1.911725,-0.826130,-1.715371,-0.577091,-1.041486,0.596067,-0.363425,...,-2.895826,-0.738275,2.361818,-0.060753,0.727249,-0.271882,5.232157,-4.218259,-2.724883,-0.063775
2,0.256023,-1.059874,NaN,0.345678,1.513814,1.243864,-0.509648,-0.800481,-0.115945,0.595777,...,2.252834,0.472496,2.491386,0.353381,-0.260682,-0.000833,-0.116457,-2.131747,3.661499,-0.131576
3,-0.728420,-2.432399,-2.453602,-0.020509,0.333397,0.086049,-1.787601,0.667011,0.761564,-2.217847,...,2.004600,-4.664806,-0.847211,-0.264249,0.664334,-0.557868,8.499483,-4.738799,-3.054611,0.494152
4,0.590212,-0.066127,0.468009,-1.096038,0.119399,-1.809710,0.466358,-0.053196,-0.580320,-1.143500,...,0.976937,2.558883,3.377724,0.846891,0.696032,0.554121,-5.979714,-2.869631,3.733057,-0.722943
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,-0.823740,0.285673,0.343307,-0.436747,1.700549,-1.069432,0.819698,-0.168457,-0.429074,0.844075,...,1.799592,-0.301352,5.339675,-0.991529,1.279494,-0.841051,-2.276500,1.762961,5.324553,-0.228733
999996,-0.769106,-0.387363,-1.227469,0.601183,0.351161,0.219475,-0.530277,0.853452,0.608646,1.648023,...,1.909697,-1.299360,-0.071713,-0.162173,0.072501,-0.614687,-1.265524,0.190385,-0.344112,-0.346807
999997,0.147534,-0.715276,-0.465049,-1.988941,-1.594535,-1.044882,3.159455,-0.634108,0.890382,-1.212444,...,2.891854,3.105002,-3.470520,NaN,0.096988,0.569255,3.609790,-0.584108,-1.492096,-0.997502


In [3]:
def cols_by_prefix(columns, prefix):
    return [x for x in columns if x.startswith(prefix)]

cols_f1 = cols_by_prefix(data.columns, 'F_1')
cols_f2 = cols_by_prefix(data.columns, 'F_2')
cols_f3 = cols_by_prefix(data.columns, 'F_3')
cols_f4 = cols_by_prefix(data.columns, 'F_4')
cols_f134 = cols_f1 + cols_f3 + cols_f4
cols_f123 = cols_f1 + cols_f2 + cols_f3

data_f134 = data[cols_f134]
data_f1 = data[cols_f1]
data_f2 = data[cols_f2]
data_f3 = data[cols_f3]
data_f4 = data[cols_f4]

In [4]:
def make_training(df, n, p, random_state):
    # first find all rows with *no* NaN; sample n rows
    df = df[~df.isnull().any(axis=1)]
    if n > 0:
        df = df.sample(n=n, random_state=random_state)
    
    # random mask of NaN locations; only cols F_1*, F_3*, F_4*
    mask = np.random.random(df[cols_f134].shape) < p
    df_na = df[cols_f134].mask(mask)

    # put it back together with F_2*
    df_na = pd.concat([df_na[cols_f1], df[cols_f2], df_na[cols_f3], df_na[cols_f4]], axis=1)
    return df, df_na, df_na.isna().sum().sum()

def sse_cols(df1, df2):
    return ((df1 - df2).pow(2)).sum()

def rmse(df1, df2, n):
    return (sse_cols(df1, df2).sum()/n)**0.5

In [5]:
%%time

p = 1/55

train, train_na, na_count = make_training(data, -1, p, RANDOM_STATE)
train_raw = train_na.copy(deep=True)

imputer = SimpleImputer(strategy="mean")
train_na[:] = imputer.fit_transform(train_na)
print(f'RMSE={rmse(train, train_na, na_count)}')

RMSE=1.4181785452698477
CPU times: total: 7.23 s
Wall time: 6.57 s


In [6]:
train_isna = train_raw.isna().melt(value_name="na", ignore_index=False)
train_na_melted = train_na.melt(ignore_index=False)
display(train_isna[train_isna['na']])
display(train_na_melted)


,variable,na
row_id,,
229,F_1_0,True
817,F_1_0,True
826,F_1_0,True
878,F_1_0,True
915,F_1_0,True
...,...,...
998985,F_4_14,True
999137,F_4_14,True
999313,F_4_14,True


,variable,value
row_id,,
6,F_1_0,0.533210
8,F_1_0,-0.334935
9,F_1_0,0.144170
15,F_1_0,0.386307
16,F_1_0,0.641497
...,...,...
999989,F_4_14,-1.961221
999995,F_4_14,-0.228733
999996,F_4_14,-0.346807


In [7]:
df1 = train_isna.reset_index().set_index(["row_id", "variable"]).sort_index()
df2 = train_na_melted.reset_index().set_index(["row_id", "variable"]).sort_index()
df2 

value
row_id variable          
6      F_1_0     0.533210
       F_1_1     1.662285
       F_1_10    2.267846
       F_1_11    0.605591
       F_1_12   -0.838225
...                   ...
999999 F_4_5     1.079820
       F_4_6    -1.098772
       F_4_7    -1.428362
       F_4_8    -1.255175
       F_4_9     0.509799

[29181920 rows x 1 columns]

## Use `row-col` from the sample submission to find the imputed values

In [10]:
for i in tqdm(submission.index):
    row = int(i.split('-')[0])
    col = i.split('-')[1]
    submission.loc[i, 'value'] = data.loc[row, col]

submission.to_csv('submission.csv')

100%|█████████████████████████████████████████████████████████████████████| 1000000/1000000 [00:59<00:00, 16872.01it/s]


In [11]:
mean_imputer = SimpleImputer(strategy="mean")

imputer = ColumnTransformer(
    transformers=[
        ("mean1", mean_imputer, cols_f1),
        ("mean2", mean_imputer, cols_f2),
        ("mean3", mean_imputer, cols_f3),
    ],
    remainder='passthrough'
)

train, train_na, na_count = make_training(data, -1, p, RANDOM_STATE)
train_na[:] = imputer.fit_transform(train_na)

print(f'RMSE={rmse(train, train_na, na_count)}')


RMSE=0.8266835378390248


In [12]:
mean_imputer = SimpleImputer(strategy="mean")

imputer = ColumnTransformer(
    transformers=[
        ("mean1", mean_imputer, cols_f1),
        ("mean2", mean_imputer, cols_f2),
        ("mean3", mean_imputer, cols_f3),
    ],
    remainder='passthrough'
)

train, train_na, na_count = make_training(data, -1, p, RANDOM_STATE)
train_na[:] = imputer.fit_transform(train_na)


In [13]:
%%time

imputer = IterativeImputer(estimator=XGBRegressor(n_estimators=2000, 
                                                  tree_method='gpu_hist',
                                                  random_state=RANDOM_STATE), 
                                     max_iter=2,
                                     verbose=2,
                                    random_state=RANDOM_STATE)
train_na[cols_f4] = imputer.fit_transform(train_na[cols_f4])
print(f'RMSE={rmse(train, train_na, na_count)}')

[IterativeImputer] Completing matrix with shape (364774, 15)
[IterativeImputer] Ending imputation round 1/2, elapsed time 364.52
[IterativeImputer] Change: 24.62562176243649, scaled tolerance: 0.029931176 
[IterativeImputer] Ending imputation round 2/2, elapsed time 731.16
[IterativeImputer] Change: 8.90385627746582, scaled tolerance: 0.029931176 


C:\Users\orovi\anaconda3\envs\kaggle\lib\site-packages\sklearn\impute\_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


RMSE=0.8948626592575114
CPU times: total: 13min 19s
Wall time: 12min 12s


In [14]:
#RMSE=0.8951881644153129

In [15]:
mean_imputer = SimpleImputer(strategy="mean")

imputer = ColumnTransformer(
    transformers=[
        ("mean1", mean_imputer, cols_f1),
        ("mean2", mean_imputer, cols_f2),
        ("mean3", mean_imputer, cols_f3),
    ],
    remainder='passthrough'
)

data[:] = imputer.fit_transform(data)

In [16]:
%%time
imputer = IterativeImputer(estimator=XGBRegressor(n_estimators=2000, 
                                                  max_depth=8,
                                                  tree_method='gpu_hist',
                                                  random_state=RANDOM_STATE), 
                                     max_iter=2,
                                     verbose=2,
                                    random_state=RANDOM_STATE)
df = data[cols_f4]
df = pd.concat([df, df.isna()], axis=1)
display(df)
dft = imputer.fit_transform(df)
data[cols_f4] = dft[:,0:15]
display(data)

,F_4_0,F_4_1,F_4_2,F_4_3,F_4_4,F_4_5,F_4_6,F_4_7,F_4_8,F_4_9,...,F_4_5,F_4_6,F_4_7,F_4_8,F_4_9,F_4_10,F_4_11,F_4_12,F_4_13,F_4_14
row_id,,,,,,,,,,,,,,,,,,,,,
0,5.547214,1.066871,-0.134313,-0.101040,-0.660871,3.744152,0.794438,0.265185,-0.561809,0.196480,...,False,False,False,False,False,False,False,False,False,False
1,-1.707374,-1.188114,-0.562419,-1.462988,1.290672,-2.895826,-0.738275,2.361818,-0.060753,0.727249,...,False,False,False,False,False,False,False,False,False,False
2,1.914908,3.877128,NaN,0.358635,0.443973,2.252834,0.472496,2.491386,0.353381,-0.260682,...,False,False,False,False,False,False,False,False,False,False
3,-2.638262,0.546676,0.865400,-0.857077,2.667105,2.004600,-4.664806,-0.847211,-0.264249,0.664334,...,False,False,False,False,False,False,False,False,False,False
4,-0.230342,-0.459019,1.128705,-0.748683,NaN,0.976937,2.558883,3.377724,0.846891,0.696032,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,1.148096,-5.126425,0.746223,-0.195402,2.615170,1.799592,-0.301352,5.339675,-0.991529,1.279494,...,False,False,False,False,False,False,False,False,False,False
999996,-4.990146,-1.636969,0.862797,0.331960,2.386669,1.909697,-1.299360,-0.071713,-0.162173,0.072501,...,False,False,False,False,False,False,False,False,False,False
999997,1.067439,0.293753,-0.866815,-0.291116,1.208282,2.891854,3.105002,-3.470520,NaN,0.096988,...,False,False,False,True,False,False,False,False,False,False


[IterativeImputer] Completing matrix with shape (1000000, 30)
[IterativeImputer] Ending imputation round 1/2, elapsed time 1783.65
[IterativeImputer] Change: 24.989014803287404, scaled tolerance: 0.031229363 
[IterativeImputer] Ending imputation round 2/2, elapsed time 4860.01
[IterativeImputer] Change: 8.964101314544678, scaled tolerance: 0.031229363 


C:\Users\orovi\anaconda3\envs\kaggle\lib\site-packages\sklearn\impute\_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


,F_1_0,F_1_1,F_1_2,F_1_3,F_1_4,F_1_5,F_1_6,F_1_7,F_1_8,F_1_9,...,F_4_5,F_4_6,F_4_7,F_4_8,F_4_9,F_4_10,F_4_11,F_4_12,F_4_13,F_4_14
row_id,,,,,,,,,,,,,,,,,,,,,
0,-0.354591,-0.464038,2.304115,0.734486,1.696395,0.136285,-0.518344,0.502640,-1.852504,-0.500665,...,3.744152,0.794438,0.265185,-0.561809,0.196480,0.373434,6.206995,3.809505,1.236486,1.182055
1,1.380940,-0.499626,-0.418548,1.911725,-0.826130,-1.715371,-0.577091,-1.041486,0.596067,-0.363425,...,-2.895826,-0.738275,2.361818,-0.060753,0.727249,-0.271882,5.232157,-4.218259,-2.724883,-0.063775
2,0.256023,-1.059874,0.000551,0.345678,1.513814,1.243864,-0.509648,-0.800481,-0.115945,0.595777,...,2.252834,0.472496,2.491386,0.353381,-0.260682,-0.000833,-0.116457,-2.131747,3.661499,-0.131576
3,-0.728420,-2.432399,-2.453602,-0.020509,0.333397,0.086049,-1.787601,0.667011,0.761564,-2.217847,...,2.004600,-4.664806,-0.847211,-0.264249,0.664334,-0.557868,8.499483,-4.738799,-3.054611,0.494152
4,0.590212,-0.066127,0.468009,-1.096038,0.119399,-1.809710,0.466358,-0.053196,-0.580320,-1.143500,...,0.976937,2.558883,3.377724,0.846891,0.696032,0.554121,-5.979714,-2.869631,3.733057,-0.722943
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,-0.823740,0.285673,0.343307,-0.436747,1.700549,-1.069432,0.819698,-0.168457,-0.429074,0.844075,...,1.799592,-0.301352,5.339675,-0.991529,1.279494,-0.841051,-2.276500,1.762961,5.324553,-0.228733
999996,-0.769106,-0.387363,-1.227469,0.601183,0.351161,0.219475,-0.530277,0.853452,0.608646,1.648023,...,1.909697,-1.299360,-0.071713,-0.162173,0.072501,-0.614687,-1.265524,0.190385,-0.344112,-0.346807
999997,0.147534,-0.715276,-0.465049,-1.988941,-1.594535,-1.044882,3.159455,-0.634108,0.890382,-1.212444,...,2.891854,3.105002,-3.470520,-0.084705,0.096988,0.569255,3.609790,-0.584108,-1.492096,-0.997502


CPU times: total: 1h 25min 40s
Wall time: 1h 21min 6s


In [17]:
dft.shape

(1000000, 30)

In [18]:
dft[:,0:15].shape

(1000000, 15)